In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [2]:
%cd F:\UBC_Okanagan\Courses\DATA 553 Privacy, Security, and Professional Ethics\data_553\project\REJ_data\REJ_data

[Errno 2] No such file or directory: 'F:UBC_OkanaganCoursesDATA 553 Privacy, Security, and Professional Ethicsdata_553projectREJ_dataREJ_data'
/Users/kt/Documents/UBC_MDS/block4/553/553proj/553project/methods


In [3]:
import os
os.chdir('/Users/kt/Documents/UBC_MDS/block4/553/553proj/553project/methods/data/')

### Load Training and Testing data

In [4]:
# training data
with open('/Users/kt/Documents/UBC_MDS/block4/553/553proj/553project/methods/data/REJ_data/Feature_tt.json', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    #alljson=json_data[0:400]

In [5]:
df=pd.DataFrame.from_dict(json_normalize(json_data), orient='columns')
df.shape

(616, 24)

In [6]:
# Testing data
p=pd.read_csv("/Users/kt/Documents/UBC_MDS/block4/553/553proj/553project/methods/data/Dataset3.csv")

### Train Classifier On Training Data

In [7]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [8]:
CUSTOM_STOPWORDS = ['i', 'me','up','my', 'myself', 'we', 'our', 'ours',
                    'ourselves', 'you', 'your', 'yours','yourself', 'yourselves',
                    'he', 'him', 'his', 'himself', 'she', 'her', 'hers' ,'herself',
                    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs',
                    'themselves' ,'am', 'is', 'are','a', 'an', 'the', 'and','in',
                    'out', 'on','up','down', 's', 't']

In [9]:
X_t=df['comment']
y_t=df['label']
tfidf_transformer = TfidfTransformer()

##### Classifier 1: Using [tfidf,  Lemmatizing, removing stop_words, bigram classification] techniques:

In [10]:
count_vect = CountVectorizer(tokenizer=LemmaTokenizer(),stop_words =frozenset(CUSTOM_STOPWORDS),ngram_range=(2, 2))
X_train_counts = count_vect.fit_transform(X_t)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_t)

##### Classifier 2:  Using [tfidf] Classification Technique:

In [11]:
count_vect2=CountVectorizer() #tfidf
X_train_counts2 = count_vect2.fit_transform(X_t)
X_train_tfidf2 = tfidf_transformer.fit_transform(X_train_counts2)
clf2= MultinomialNB().fit(X_train_tfidf2, y_t)

##### Classifier 3:  Using [tfidf, Lemmatizing] Classification Technique:

In [12]:
count_vect3=CountVectorizer(tokenizer=LemmaTokenizer())
X_train_counts3 = count_vect3.fit_transform(X_t)
X_train_tfidf3 = tfidf_transformer.fit_transform(X_train_counts3)
clf3= MultinomialNB().fit(X_train_tfidf3, y_t)

### Using Classifier to predict on Testing data

##### Prediction on Classifier 1

In [13]:
predicted1=clf.predict(count_vect.transform([i for i in p['processed_text']]))
predicted1=pd.DataFrame(predicted1)

In [14]:
reviewed = pd.read_csv('/Users/kt/Documents/UBC_MDS/block4/553/553proj/553project/methods/data/labelled.csv')
reviewed

,processed_text,Rating,User Experience,Bug Report,Feature Report,label
0,A great game to be during times Nice,1,0,0,0,Rating
1,I truly love pandora,0,1,0,0,UserExperience
2,I need all the help I need 5ft and I need to l...,0,1,0,0,UserExperience
3,it s not great but it s good,1,0,0,0,Rating
4,I think this is cool this is learning and and ...,0,1,0,0,UserExperience
...,...,...,...,...,...,...
379,Really good for,1,0,0,0,Rating
380,It is my first time to use this see who it is,0,0,0,0,UserExperience
381,I love it,1,0,0,0,Rating
382,fun fact it s fun,1,0,0,0,Rating


In [15]:
from sklearn.metrics import *
import csv


y_true = reviewed['label']
y_pred = predicted1

f1 = precision_recall_fscore_support(y_true, y_pred, average = "macro")


//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


##### Prediction on Classifier 2

In [16]:
predicted2=clf2.predict(count_vect2.transform([i for i in p['processed_text']]))
predicted2=pd.DataFrame(predicted2)


y_true = reviewed['label']
y_pred = predicted2


f2 = precision_recall_fscore_support(y_true, y_pred, average = "macro")


##### Prediction on Classifier 3

In [17]:
predicted3=clf3.predict(count_vect3.transform([i for i in p['processed_text']]))
predicted3=pd.DataFrame(predicted3)

y_true = reviewed['label']
y_pred = predicted3

f3 = precision_recall_fscore_support(y_true, y_pred, average = "macro")

In [18]:
pd.DataFrame({'technique' : ['tfidf, Lemmatizing, removing stop_words, bigram', 'tf-idf', 'tf-idf+lemmatization'], 
             'prec' : [f1[0], f2[0], f3[0]],
             'recall': [f1[1], f2[1], f3[1]],
             'fscore': [f1[2], f2[2], f3[2]]})

,technique,prec,recall,fscore
0,"tfidf, Lemmatizing, removing stop_words, bigram",0.033333,0.153333,0.054762
1,tf-idf,0.027184,0.186667,0.047458
2,tf-idf+lemmatization,0.025926,0.186667,0.045528
